In [1]:
import pandas as pd
import numpy as np

import boto3
import io

import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler


# Set the maximum number of rows and Columns to 200
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)

In [2]:
# Create function to load s3 files from AWS
def load_s3_file(file_name, s3, bucket_name, df_name):
    import boto3
    # Downloading the file from S3 and parsing it into a dataframe
    s3.download_file(bucket_name, file_name, file_name)
    
    # Low memory is set to False because columns have mixed data types
    df_name = pd.read_csv(file_name, index_col=False, low_memory=False)
    return df_name 

In [3]:
# Load files from AWS
s3R = boto3.resource("s3")
s3 = boto3.client("s3")
bucket_name = "capstonehaystacks"

# Listing the contents of the S3 bucket
for obj in s3R.Bucket(bucket_name).objects.all():
    print(obj.key)

ACSDP5Y2021.DP04-Data.csv
ACSST5Y2021.S0101-Data.csv
ACSST5Y2021.S0801-Data.csv
ACSST5Y2021.S1901-Data.csv
GA_LISTINGS_SALES.csv
GA_LISTINGS_SALES_V2.csv
GA_listing.csv
POI_second_tab.csv
all_zips_grocery_store.json
all_zips_restaurant.json
atlanta-geo.csv
atlanta_cbsa_zip.csv
atlanta_hdma_2021.csv
atlanta_hdma_census_2021.csv
atlanta_hdma_tract_2021.csv
atlanta_listings.csv
census_all.csv
census_all_perCapita.csv
census_data
core_geo_dataset.csv
crime.csv
crime_rating_zipcode.csv
elementary_schools.csv
ga_georgia_zip_codes_geo.min.json
high_schools.csv
hmda_2017_ga_all-records_labels.csv
hmda_2021_ga_all-records_labels.csv
jason_listing.csv
listing-with-poi-distances.csv
listings_with_tract.csv
middle_schools.csv
poi-google-exploded.csv
poi_combined_haystack.csv
poi_combined_haystack_ALL.csv
poi_combined_haystack_ALL_CLEANED.csv
poi_expanded.csv
points-of-interest-google.csv
points-of-interest-google2.csv
points-of-interest-google3.csv
points-of-interest-haystacks.csv
schools.csv
zip_

In [4]:
# Load S3 files to variables
census = load_s3_file("census_all.csv", s3, bucket_name, df_name="census")
census_capita = load_s3_file("census_all_perCapita.csv", s3, bucket_name, df_name="census_capita")
hdma = load_s3_file("atlanta_hdma_2021.csv", s3, bucket_name, df_name="hdma")

In [5]:
hdma.columns

Index(['zip_code', 'average_loan_amount_home_improvement_approved',
       'average_loan_amount_home_improvement_denied',
       'average_loan_amount_home_purchase_approved',
       'average_loan_amount_home_purchase_denied',
       'loan_count_home_improvement_approved',
       'loan_count_home_improvement_denied',
       'loan_count_home_purchase_approved', 'loan_count_home_purchase_denied',
       'total_loan_count', 'total_approved_loans', 'total_denied_loans',
       'approval_percentage', 'zip_median_income', 'population',
       'zip_owner_occupied_units', 'total_one_to_four_family_homes',
       'median_age_of_housing_units'],
      dtype='object')

In [6]:
census_capita.columns

Index(['zipcode', 'restaurant_above_4.7_percentage', 'total_workers',
       'car_commute', 'travel_less_10', 'travel_10_14', 'travel_15_19',
       'travel_20_24', 'travel_25_29', 'travel_30_34',
       ...
       'window_tinting_service_per_Capita',
       'window_treatment_store_per_Capita', 'wine_bar_per_Capita',
       'wine_store_per_Capita', 'womens_clothing_store_per_Capita',
       'womens_health_clinic_per_Capita',
       'wood_floor_installation_service_per_Capita', 'yoga_studio_per_Capita',
       'restaurant_above_4.7_per_Capita', 'restaurant_below_4.7_per_Capita'],
      dtype='object', length=800)

### DROP some Census Columns

In [7]:
census_capita.columns[:50]

Index(['zipcode', 'restaurant_above_4.7_percentage', 'total_workers',
       'car_commute', 'travel_less_10', 'travel_10_14', 'travel_15_19',
       'travel_20_24', 'travel_25_29', 'travel_30_34', 'travel_35_44',
       'travel_45_59', 'travel_more_60', 'travel_mean', 'population',
       'percent_male', 'percent_under_15', 'percent_teen_15_19',
       'percent_college_20_24', 'percent_25_39', 'percent_40-59',
       'percent_over_60', 'total_units', 'rental_vacancy_rate',
       'median_homeowner_value', 'median_rental_value',
       'percent_owner_occupied', 'percent_after_2019',
       'rent_less_15_percent_income', 'rent_over_30_percent',
       'rent_15_30_percent', 'rent_less_999', 'rent_1000_2500',
       'rent_over_2500', 'gross_rental_yield', 'total_households',
       'percent_less_10k', 'percent_10k_15k', 'percent_15k_25k',
       'percent_25k_35k', 'percent_35k_50k', 'percent_50k_75k',
       'percent_75k_100k', 'percent_100k_150k', 'percent_150k_200k',
       'percent_more

In [8]:
drop_cols = ['total_workers', 'total_households', 'household_median_income',
             'restaurant_above_4.7_percentage', 'restaurant_above_4.7_per_Capita', 'restaurant_below_4.7_per_Capita']
census_capita = census_capita.drop(drop_cols, axis=1)

### Standardizing Values in Census Data/POI

In [9]:
# Select the column to exclude
column_to_exclude = 'zipcode'

df_to_standardize = census_capita.drop(column_to_exclude, axis=1)
scaler = StandardScaler()
standardized_values = scaler.fit_transform(df_to_standardize)
census_capita_standardized = pd.DataFrame(standardized_values, columns=df_to_standardize.columns)

# Add the excluded column back to the standardized DataFrame
census_capita_standardized[column_to_exclude] = census_capita[column_to_exclude]

# Pop the excluded column and insert it at the first position
zipcode_column = census_capita_standardized.pop(column_to_exclude)
census_capita_standardized.insert(0, column_to_exclude, zipcode_column)

census_capita_standardized.head()

,zipcode,car_commute,travel_less_10,travel_10_14,travel_15_19,travel_20_24,travel_25_29,travel_30_34,travel_35_44,travel_45_59,travel_more_60,travel_mean,population,percent_male,percent_under_15,percent_teen_15_19,percent_college_20_24,percent_25_39,percent_40-59,percent_over_60,total_units,rental_vacancy_rate,median_homeowner_value,median_rental_value,percent_owner_occupied,percent_after_2019,rent_less_15_percent_income,rent_over_30_percent,rent_15_30_percent,rent_less_999,rent_1000_2500,rent_over_2500,gross_rental_yield,percent_less_10k,percent_10k_15k,percent_15k_25k,percent_25k_35k,percent_35k_50k,percent_50k_75k,percent_75k_100k,percent_100k_150k,percent_150k_200k,percent_more_200k,household_mean_income,accountant_per_Capita,addiction_treatment_center_per_Capita,advertising_agency_per_Capita,after_school_program_per_Capita,air_conditioning_contractor_per_Capita,air_conditioning_repair_service_per_Capita,air_conditioning_system_supplier_per_Capita,air_duct_cleaning_service_per_Capita,allergist_per_Capita,alternative_fuel_station_per_Capita,alternative_medicine_practitioner_per_Capita,american_restaurant_per_Capita,amusement_center_per_Capita,animal_control_service_per_Capita,animal_hospital_per_Capita,antique_furniture_store_per_Capita,antique_store_per_Capita,apartment_building_per_Capita,apartment_complex_per_Capita,apartment_rental_agency_per_Capita,appliance_repair_service_per_Capita,appliance_store_per_Capita,art_center_per_Capita,art_gallery_per_Capita,asian_fusion_restaurant_per_Capita,asian_restaurant_per_Capita,assisted_living_facility_per_Capita,association_or_organization_per_Capita,atm_per_Capita,attorney_per_Capita,audiologist_per_Capita,auto_air_conditioning_service_per_Capita,auto_body_parts_supplier_per_Capita,auto_body_shop_per_Capita,auto_broker_per_Capita,auto_dent_removal_service_per_Capita,auto_electrical_service_per_Capita,auto_glass_shop_per_Capita,auto_insurance_agency_per_Capita,auto_parts_store_per_Capita,auto_radiator_repair_service_per_Capita,auto_repair_shop_per_Capita,auto_restoration_service_per_Capita,auto_tune_up_service_per_Capita,auto_wrecker_per_Capita,automobile_storage_facility_per_Capita,baby_clothing_store_per_Capita,baby_store_per_Capita,bagel_shop_per_Capita,bail_bonds_service_per_Capita,bakery_per_Capita,bank_per_Capita,bankruptcy_attorney_per_Capita,banner_store_per_Capita,banquet_hall_per_Capita,baptist_church_per_Capita,bar_per_Capita,bar_&_grill_per_Capita,barbecue_restaurant_per_Capita,barber_shop_per_Capita,barber_supply_store_per_Capita,baseball_field_per_Capita,bathroom_remodeler_per_Capita,bathroom_supply_store_per_Capita,beauty_salon_per_Capita,beauty_supply_store_per_Capita,bedding_store_per_Capita,bedroom_furniture_store_per_Capita,beer_store_per_Capita,bicycle_repair_shop_per_Capita,bicycle_shop_per_Capita,bistro_per_Capita,boat_dealer_per_Capita,boat_ramp_per_Capita,boat_repair_shop_per_Capita,boat_storage_facility_per_Capita,body_piercing_shop_per_Capita,book_store_per_Capita,bookkeeping_service_per_Capita,boot_store_per_Capita,boutique_per_Capita,brake_shop_per_Capita,breakfast_restaurant_per_Capita,brewery_per_Capita,bridal_shop_per_Capita,brunch_restaurant_per_Capita,bubble_tea_store_per_Capita,buffet_restaurant_per_Capita,building_materials_store_per_Capita,building_materials_supplier_per_Capita,burrito_restaurant_per_Capita,bus_stop_per_Capita,business_center_per_Capita,business_management_consultant_per_Capita,business_park_per_Capita,business_to_business_service_per_Capita,butcher_shop_per_Capita,cabinet_maker_per_Capita,cabinet_store_per_Capita,cable_company_per_Capita,cafe_per_Capita,cajun_restaurant_per_Capita,cake_shop_per_Capita,camp_per_Capita,campground_per_Capita,candle_store_per_Capita,candy_store_per_Capita,car_accessories_store_per_Capita,car_battery_store_per_Capita,car_dealer_per_Capita,car_detailing_service_per_Capita,car_finance_and_loan_company_per_Capita,car_inspection_station_per_Capita,car_leasing_service_per_Capita,car_rental_agency_per_Ca

#### Reverse Standardization (not neccesary)

In [10]:
reversed_values = scaler.inverse_transform(standardized_values)

# Create a new DataFrame with reversed values
census_capita_reversed = pd.DataFrame(reversed_values, columns=df_to_standardize.columns)

# Add the excluded column back to the reversed DataFrame
census_capita_reversed[column_to_exclude] = census_capita[column_to_exclude]

# Pop the excluded column and insert it at the first position
zipcode_column = census_capita_reversed.pop(column_to_exclude)
census_capita_reversed.insert(0, column_to_exclude, zipcode_column)

census_capita_reversed.head()

,zipcode,car_commute,travel_less_10,travel_10_14,travel_15_19,travel_20_24,travel_25_29,travel_30_34,travel_35_44,travel_45_59,travel_more_60,travel_mean,population,percent_male,percent_under_15,percent_teen_15_19,percent_college_20_24,percent_25_39,percent_40-59,percent_over_60,total_units,rental_vacancy_rate,median_homeowner_value,median_rental_value,percent_owner_occupied,percent_after_2019,rent_less_15_percent_income,rent_over_30_percent,rent_15_30_percent,rent_less_999,rent_1000_2500,rent_over_2500,gross_rental_yield,percent_less_10k,percent_10k_15k,percent_15k_25k,percent_25k_35k,percent_35k_50k,percent_50k_75k,percent_75k_100k,percent_100k_150k,percent_150k_200k,percent_more_200k,household_mean_income,accountant_per_Capita,addiction_treatment_center_per_Capita,advertising_agency_per_Capita,after_school_program_per_Capita,air_conditioning_contractor_per_Capita,air_conditioning_repair_service_per_Capita,air_conditioning_system_supplier_per_Capita,air_duct_cleaning_service_per_Capita,allergist_per_Capita,alternative_fuel_station_per_Capita,alternative_medicine_practitioner_per_Capita,american_restaurant_per_Capita,amusement_center_per_Capita,animal_control_service_per_Capita,animal_hospital_per_Capita,antique_furniture_store_per_Capita,antique_store_per_Capita,apartment_building_per_Capita,apartment_complex_per_Capita,apartment_rental_agency_per_Capita,appliance_repair_service_per_Capita,appliance_store_per_Capita,art_center_per_Capita,art_gallery_per_Capita,asian_fusion_restaurant_per_Capita,asian_restaurant_per_Capita,assisted_living_facility_per_Capita,association_or_organization_per_Capita,atm_per_Capita,attorney_per_Capita,audiologist_per_Capita,auto_air_conditioning_service_per_Capita,auto_body_parts_supplier_per_Capita,auto_body_shop_per_Capita,auto_broker_per_Capita,auto_dent_removal_service_per_Capita,auto_electrical_service_per_Capita,auto_glass_shop_per_Capita,auto_insurance_agency_per_Capita,auto_parts_store_per_Capita,auto_radiator_repair_service_per_Capita,auto_repair_shop_per_Capita,auto_restoration_service_per_Capita,auto_tune_up_service_per_Capita,auto_wrecker_per_Capita,automobile_storage_facility_per_Capita,baby_clothing_store_per_Capita,baby_store_per_Capita,bagel_shop_per_Capita,bail_bonds_service_per_Capita,bakery_per_Capita,bank_per_Capita,bankruptcy_attorney_per_Capita,banner_store_per_Capita,banquet_hall_per_Capita,baptist_church_per_Capita,bar_per_Capita,bar_&_grill_per_Capita,barbecue_restaurant_per_Capita,barber_shop_per_Capita,barber_supply_store_per_Capita,baseball_field_per_Capita,bathroom_remodeler_per_Capita,bathroom_supply_store_per_Capita,beauty_salon_per_Capita,beauty_supply_store_per_Capita,bedding_store_per_Capita,bedroom_furniture_store_per_Capita,beer_store_per_Capita,bicycle_repair_shop_per_Capita,bicycle_shop_per_Capita,bistro_per_Capita,boat_dealer_per_Capita,boat_ramp_per_Capita,boat_repair_shop_per_Capita,boat_storage_facility_per_Capita,body_piercing_shop_per_Capita,book_store_per_Capita,bookkeeping_service_per_Capita,boot_store_per_Capita,boutique_per_Capita,brake_shop_per_Capita,breakfast_restaurant_per_Capita,brewery_per_Capita,bridal_shop_per_Capita,brunch_restaurant_per_Capita,bubble_tea_store_per_Capita,buffet_restaurant_per_Capita,building_materials_store_per_Capita,building_materials_supplier_per_Capita,burrito_restaurant_per_Capita,bus_stop_per_Capita,business_center_per_Capita,business_management_consultant_per_Capita,business_park_per_Capita,business_to_business_service_per_Capita,butcher_shop_per_Capita,cabinet_maker_per_Capita,cabinet_store_per_Capita,cable_company_per_Capita,cafe_per_Capita,cajun_restaurant_per_Capita,cake_shop_per_Capita,camp_per_Capita,campground_per_Capita,candle_store_per_Capita,candy_store_per_Capita,car_accessories_store_per_Capita,car_battery_store_per_Capita,car_dealer_per_Capita,car_detailing_service_per_Capita,car_finance_and_loan_company_per_Capita,car_inspection_station_per_Capita,car_leasing_service_per_Capita,car_rental_agency_per_Ca

### KNN to find Similar Profiled Zipcodes

In [11]:
# Feature Selection
selected_cols = census_capita_standardized.drop('zipcode', axis=1).columns
X = census_capita_standardized.drop('zipcode', axis=1).values 

k=6 # No. of zipcodes to consider
model = NearestNeighbors(n_neighbors=k)
model.fit(X)

NearestNeighbors(n_neighbors=6)

In [12]:
# Define my Zip Code vals
my_zip = 30002
my_zip_vals = census_capita_standardized[census_capita_standardized['zipcode'] == my_zip].iloc[:,1:]
my_zip_vals

,car_commute,travel_less_10,travel_10_14,travel_15_19,travel_20_24,travel_25_29,travel_30_34,travel_35_44,travel_45_59,travel_more_60,travel_mean,population,percent_male,percent_under_15,percent_teen_15_19,percent_college_20_24,percent_25_39,percent_40-59,percent_over_60,total_units,rental_vacancy_rate,median_homeowner_value,median_rental_value,percent_owner_occupied,percent_after_2019,rent_less_15_percent_income,rent_over_30_percent,rent_15_30_percent,rent_less_999,rent_1000_2500,rent_over_2500,gross_rental_yield,percent_less_10k,percent_10k_15k,percent_15k_25k,percent_25k_35k,percent_35k_50k,percent_50k_75k,percent_75k_100k,percent_100k_150k,percent_150k_200k,percent_more_200k,household_mean_income,accountant_per_Capita,addiction_treatment_center_per_Capita,advertising_agency_per_Capita,after_school_program_per_Capita,air_conditioning_contractor_per_Capita,air_conditioning_repair_service_per_Capita,air_conditioning_system_supplier_per_Capita,air_duct_cleaning_service_per_Capita,allergist_per_Capita,alternative_fuel_station_per_Capita,alternative_medicine_practitioner_per_Capita,american_restaurant_per_Capita,amusement_center_per_Capita,animal_control_service_per_Capita,animal_hospital_per_Capita,antique_furniture_store_per_Capita,antique_store_per_Capita,apartment_building_per_Capita,apartment_complex_per_Capita,apartment_rental_agency_per_Capita,appliance_repair_service_per_Capita,appliance_store_per_Capita,art_center_per_Capita,art_gallery_per_Capita,asian_fusion_restaurant_per_Capita,asian_restaurant_per_Capita,assisted_living_facility_per_Capita,association_or_organization_per_Capita,atm_per_Capita,attorney_per_Capita,audiologist_per_Capita,auto_air_conditioning_service_per_Capita,auto_body_parts_supplier_per_Capita,auto_body_shop_per_Capita,auto_broker_per_Capita,auto_dent_removal_service_per_Capita,auto_electrical_service_per_Capita,auto_glass_shop_per_Capita,auto_insurance_agency_per_Capita,auto_parts_store_per_Capita,auto_radiator_repair_service_per_Capita,auto_repair_shop_per_Capita,auto_restoration_service_per_Capita,auto_tune_up_service_per_Capita,auto_wrecker_per_Capita,automobile_storage_facility_per_Capita,baby_clothing_store_per_Capita,baby_store_per_Capita,bagel_shop_per_Capita,bail_bonds_service_per_Capita,bakery_per_Capita,bank_per_Capita,bankruptcy_attorney_per_Capita,banner_store_per_Capita,banquet_hall_per_Capita,baptist_church_per_Capita,bar_per_Capita,bar_&_grill_per_Capita,barbecue_restaurant_per_Capita,barber_shop_per_Capita,barber_supply_store_per_Capita,baseball_field_per_Capita,bathroom_remodeler_per_Capita,bathroom_supply_store_per_Capita,beauty_salon_per_Capita,beauty_supply_store_per_Capita,bedding_store_per_Capita,bedroom_furniture_store_per_Capita,beer_store_per_Capita,bicycle_repair_shop_per_Capita,bicycle_shop_per_Capita,bistro_per_Capita,boat_dealer_per_Capita,boat_ramp_per_Capita,boat_repair_shop_per_Capita,boat_storage_facility_per_Capita,body_piercing_shop_per_Capita,book_store_per_Capita,bookkeeping_service_per_Capita,boot_store_per_Capita,boutique_per_Capita,brake_shop_per_Capita,breakfast_restaurant_per_Capita,brewery_per_Capita,bridal_shop_per_Capita,brunch_restaurant_per_Capita,bubble_tea_store_per_Capita,buffet_restaurant_per_Capita,building_materials_store_per_Capita,building_materials_supplier_per_Capita,burrito_restaurant_per_Capita,bus_stop_per_Capita,business_center_per_Capita,business_management_consultant_per_Capita,business_park_per_Capita,business_to_business_service_per_Capita,butcher_shop_per_Capita,cabinet_maker_per_Capita,cabinet_store_per_Capita,cable_company_per_Capita,cafe_per_Capita,cajun_restaurant_per_Capita,cake_shop_per_Capita,camp_per_Capita,campground_per_Capita,candle_store_per_Capita,candy_store_per_Capita,car_accessories_store_per_Capita,car_battery_store_per_Capita,car_dealer_per_Capita,car_detailing_service_per_Capita,car_finance_and_loan_company_per_Capita,car_inspection_station_per_Capita,car_leasing_service_per_Capita,car_rental_agency_per_Capita,car

In [13]:
# Find Similar Zip Codes
new_X = my_zip_vals.values

distances, indices = model.kneighbors(new_X)
similar_zip_codes = census_capita_standardized.iloc[indices[0]]['zipcode']  

print("Similar Zip Codes:")
similar_zip_codes

Similar Zip Codes:


0      30002
70     30127
30     30054
140    30296
3      30008
114    30238
Name: zipcode, dtype: int64

In [14]:
distances

array([[ 0.        , 37.65845103, 37.71393461, 37.87213629, 38.16980615,
        38.22544416]])

In [15]:
my_sim_zips = census_capita[census_capita['zipcode'].isin(similar_zip_codes)]
my_sim_zips

,zipcode,car_commute,travel_less_10,travel_10_14,travel_15_19,travel_20_24,travel_25_29,travel_30_34,travel_35_44,travel_45_59,travel_more_60,travel_mean,population,percent_male,percent_under_15,percent_teen_15_19,percent_college_20_24,percent_25_39,percent_40-59,percent_over_60,total_units,rental_vacancy_rate,median_homeowner_value,median_rental_value,percent_owner_occupied,percent_after_2019,rent_less_15_percent_income,rent_over_30_percent,rent_15_30_percent,rent_less_999,rent_1000_2500,rent_over_2500,gross_rental_yield,percent_less_10k,percent_10k_15k,percent_15k_25k,percent_25k_35k,percent_35k_50k,percent_50k_75k,percent_75k_100k,percent_100k_150k,percent_150k_200k,percent_more_200k,household_mean_income,accountant_per_Capita,addiction_treatment_center_per_Capita,advertising_agency_per_Capita,after_school_program_per_Capita,air_conditioning_contractor_per_Capita,air_conditioning_repair_service_per_Capita,air_conditioning_system_supplier_per_Capita,air_duct_cleaning_service_per_Capita,allergist_per_Capita,alternative_fuel_station_per_Capita,alternative_medicine_practitioner_per_Capita,american_restaurant_per_Capita,amusement_center_per_Capita,animal_control_service_per_Capita,animal_hospital_per_Capita,antique_furniture_store_per_Capita,antique_store_per_Capita,apartment_building_per_Capita,apartment_complex_per_Capita,apartment_rental_agency_per_Capita,appliance_repair_service_per_Capita,appliance_store_per_Capita,art_center_per_Capita,art_gallery_per_Capita,asian_fusion_restaurant_per_Capita,asian_restaurant_per_Capita,assisted_living_facility_per_Capita,association_or_organization_per_Capita,atm_per_Capita,attorney_per_Capita,audiologist_per_Capita,auto_air_conditioning_service_per_Capita,auto_body_parts_supplier_per_Capita,auto_body_shop_per_Capita,auto_broker_per_Capita,auto_dent_removal_service_per_Capita,auto_electrical_service_per_Capita,auto_glass_shop_per_Capita,auto_insurance_agency_per_Capita,auto_parts_store_per_Capita,auto_radiator_repair_service_per_Capita,auto_repair_shop_per_Capita,auto_restoration_service_per_Capita,auto_tune_up_service_per_Capita,auto_wrecker_per_Capita,automobile_storage_facility_per_Capita,baby_clothing_store_per_Capita,baby_store_per_Capita,bagel_shop_per_Capita,bail_bonds_service_per_Capita,bakery_per_Capita,bank_per_Capita,bankruptcy_attorney_per_Capita,banner_store_per_Capita,banquet_hall_per_Capita,baptist_church_per_Capita,bar_per_Capita,bar_&_grill_per_Capita,barbecue_restaurant_per_Capita,barber_shop_per_Capita,barber_supply_store_per_Capita,baseball_field_per_Capita,bathroom_remodeler_per_Capita,bathroom_supply_store_per_Capita,beauty_salon_per_Capita,beauty_supply_store_per_Capita,bedding_store_per_Capita,bedroom_furniture_store_per_Capita,beer_store_per_Capita,bicycle_repair_shop_per_Capita,bicycle_shop_per_Capita,bistro_per_Capita,boat_dealer_per_Capita,boat_ramp_per_Capita,boat_repair_shop_per_Capita,boat_storage_facility_per_Capita,body_piercing_shop_per_Capita,book_store_per_Capita,bookkeeping_service_per_Capita,boot_store_per_Capita,boutique_per_Capita,brake_shop_per_Capita,breakfast_restaurant_per_Capita,brewery_per_Capita,bridal_shop_per_Capita,brunch_restaurant_per_Capita,bubble_tea_store_per_Capita,buffet_restaurant_per_Capita,building_materials_store_per_Capita,building_materials_supplier_per_Capita,burrito_restaurant_per_Capita,bus_stop_per_Capita,business_center_per_Capita,business_management_consultant_per_Capita,business_park_per_Capita,business_to_business_service_per_Capita,butcher_shop_per_Capita,cabinet_maker_per_Capita,cabinet_store_per_Capita,cable_company_per_Capita,cafe_per_Capita,cajun_restaurant_per_Capita,cake_shop_per_Capita,camp_per_Capita,campground_per_Capita,candle_store_per_Capita,candy_store_per_Capita,car_accessories_store_per_Capita,car_battery_store_per_Capita,car_dealer_per_Capita,car_detailing_service_per_Capita,car_finance_and_loan_company_per_Capita,car_inspection_station_per_Capita,car_leasing_service_per_Capita,car_rental_agency_per_Ca

### Cosine Similarity for Census, HDMA, and POI data on selected ZIPCODE

In [16]:
# Define Column subsets
census_col_names = my_sim_zips.iloc[:,1:20].columns.tolist()
hdma_col_names = my_sim_zips.iloc[:,20:43].columns.tolist()
poi_col_names = my_sim_zips.iloc[:,44:].columns.tolist()

In [17]:
census_sim = ['zipcode']
hdma_sim = ['zipcode']
poi_sim = ['zipcode']

# Loop through each column name subset
for col_names, sim_list in zip([census_col_names, hdma_col_names, poi_col_names], [census_sim, hdma_sim, poi_sim]):
    # Select the subset of columns
    selected_cols = my_sim_zips[col_names]
    similar_rows = selected_cols.iloc[1:]

    # Calculate pairwise similarity matrix for the selected rows using cosine similarity
    similarity_matrix = cosine_similarity(similar_rows.T)
    similarity_df = pd.DataFrame(similarity_matrix, index=selected_cols.columns, columns=similar_rows.columns)

    # Sort the columns based on similarity scores
    sorted_columns = similarity_df.mean(axis=1).sort_values(ascending=False)
    top_similar_cols = sorted_columns[:5].index.tolist()

    # Assign the top similar columns to the respective list
    sim_list.extend(top_similar_cols)


print("Top similar columns for the Census subset:")
print(census_sim)
print("\n")

print("Top similar columns for the HDMA subset:")
print(hdma_sim)
print("\n")

print("Top similar columns for the POI subset:")
print(poi_sim)
print("\n")

Top similar columns for the Census subset:
['zipcode', 'percent_male', 'car_commute', 'percent_40-59', 'travel_mean', 'percent_under_15']


Top similar columns for the HDMA subset:
['zipcode', 'rent_15_30_percent', 'median_rental_value', 'percent_50k_75k', 'percent_owner_occupied', 'percent_75k_100k']


Top similar columns for the POI subset:
['zipcode', 'gas_station_per_Capita', 'self-storage_facility_per_Capita', 'pizza_restaurant_per_Capita', 'grocery_store_per_Capita', 'takeout_restaurant_per_Capita']




In [18]:
my_sim_zips[census_sim]

,zipcode,percent_male,car_commute,percent_40-59,travel_mean,percent_under_15
0,30002,49.0,77.1,27.6,29.7,19.4
3,30008,49.0,83.3,25.8,30.5,21.3
30,30054,46.0,85.2,25.0,28.5,18.0
70,30127,48.0,84.7,32.4,35.3,18.3
114,30238,47.0,90.0,27.3,36.0,24.7
140,30296,49.0,90.9,27.8,29.8,20.2


In [19]:
my_sim_zips[hdma_sim]

,zipcode,rent_15_30_percent,median_rental_value,percent_50k_75k,percent_owner_occupied,percent_75k_100k
0,30002,23.5,933.0,10.7,56.5,12.8
3,30008,29.3,1195.0,20.2,60.2,12.0
30,30054,36.5,1187.0,19.2,83.2,12.4
70,30127,37.1,1547.0,17.9,81.0,16.1
114,30238,42.0,1214.0,24.1,58.7,15.8
140,30296,33.4,1283.0,21.0,65.5,13.9


In [20]:
my_sim_zips[poi_sim]

,zipcode,gas_station_per_Capita,self-storage_facility_per_Capita,pizza_restaurant_per_Capita,grocery_store_per_Capita,takeout_restaurant_per_Capita
0,30002,0.000000,0.000000,0.165975,0.000000,0.000000
3,30008,0.142763,0.057105,0.085658,0.171316,0.142763
30,30054,0.081340,0.000000,0.000000,0.000000,0.000000
70,30127,0.190965,0.054561,0.109123,0.136403,0.136403
114,30238,0.119144,0.023829,0.000000,0.023829,0.047658
140,30296,0.072841,0.036421,0.036421,0.072841,0.000000
